# In-Depth Clothing Brand Sentiment Analysis
## Comprehensive Exploratory Data Analysis (Google Colab Version)

**Objective:** Deep dive into clothing brand customer reviews to understand:
- Product category performance
- Sentiment trends over time
- Key themes in positive vs negative reviews
- Customer satisfaction patterns
- Detailed text analysis and insights

---

In [ ]:
# Step 1: Clone the repository to get the data
!git clone https://github.com/mrityunjay-vashisth/sentiment-analysis-capstone.git
%cd sentiment-analysis-capstone

In [ ]:
# Step 2: Install required libraries
!pip install wordcloud -q

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set style for professional visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

print("✓ Libraries imported successfully")

## 1. Load Clothing Reviews Dataset

In [ ]:
# Load train and test datasets (Colab path)
train_df = pd.read_csv('data/splits/clothing_reviews_train.csv')
test_df = pd.read_csv('data/splits/clothing_reviews_test.csv')

# Combine for full analysis
df = pd.concat([train_df, test_df], ignore_index=True)

print(f"📊 Dataset Overview:")
print(f"   Total Reviews: {len(df):,}")
print(f"   Training Set: {len(train_df):,} ({len(train_df)/len(df)*100:.1f}%)")
print(f"   Test Set: {len(test_df):,} ({len(test_df)/len(df)*100:.1f}%)")
print(f"\n   Columns: {df.columns.tolist()}")
print(f"\n   Date Range: {df['created_at'].min()} to {df['created_at'].max()}")

In [ ]:
# Display sample reviews
print("\n📝 Sample Customer Reviews:\n")
df.head(5)

In [ ]:
# Data quality check
print("🔍 Data Quality Check:\n")
print(df.info())
print("\n📊 Missing Values:")
print(df.isnull().sum())
print("\n📊 Summary Statistics:")
print(df.describe())

## 2. Product Category Analysis

In [ ]:
# Analyze product categories
category_counts = df['topic'].value_counts()

print("🏷️  Product Categories Distribution:\n")
for cat, count in category_counts.items():
    pct = (count / len(df)) * 100
    print(f"   {cat:20s}: {count:5,} reviews ({pct:5.2f}%)")

print(f"\n   Total Categories: {len(category_counts)}")

In [ ]:
# Visualize top product categories
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
top_15 = category_counts.head(15)
colors = plt.cm.Set3(np.linspace(0, 1, len(top_15)))

axes[0].barh(range(len(top_15)), top_15.values, color=colors, edgecolor='black', linewidth=1.2)
axes[0].set_yticks(range(len(top_15)))
axes[0].set_yticklabels(top_15.index)
axes[0].set_xlabel('Number of Reviews', fontweight='bold')
axes[0].set_title('Top 15 Product Categories by Review Volume', fontweight='bold', fontsize=14)
axes[0].invert_yaxis()
axes[0].grid(axis='x', alpha=0.3)

# Add value labels
for i, v in enumerate(top_15.values):
    axes[0].text(v + 50, i, f'{v:,}', va='center', fontweight='bold')

# Pie chart for top categories
top_10 = category_counts.head(10)
other_count = category_counts[10:].sum()

pie_data = list(top_10.values) + [other_count]
pie_labels = list(top_10.index) + ['Others']

wedges, texts, autotexts = axes[1].pie(pie_data, labels=pie_labels, autopct='%1.1f%%',
                                        startangle=90, colors=plt.cm.Set3(np.linspace(0, 1, 11)))
axes[1].set_title('Product Category Distribution (Top 10 + Others)', fontweight='bold', fontsize=14)

# Make percentage text bold
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(9)

plt.tight_layout()
plt.show()

print("✅ Visualization complete!")

## 3. Sentiment Distribution Analysis

In [ ]:
# Overall sentiment distribution
sentiment_counts = df['label'].value_counts().reindex(['negative', 'neutral', 'positive'])

print("💭 Overall Sentiment Distribution:\n")
for sent, count in sentiment_counts.items():
    pct = (count / len(df)) * 100
    print(f"   {sent.capitalize():12s}: {count:5,} reviews ({pct:5.2f}%)")

print(f"\n   ⚠️  Class Imbalance Ratio: {sentiment_counts['positive'] / sentiment_counts['negative']:.2f}:1 (Positive:Negative)")

In [ ]:
# Visualize overall sentiment
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

colors = ['#ef5350', '#ffa726', '#66bb6a']

# Bar chart
axes[0].bar(sentiment_counts.index, sentiment_counts.values, color=colors, 
           edgecolor='black', linewidth=1.5, width=0.6)
axes[0].set_xlabel('Sentiment', fontweight='bold')
axes[0].set_ylabel('Number of Reviews', fontweight='bold')
axes[0].set_title('Clothing Reviews Sentiment Distribution', fontweight='bold', fontsize=14)
axes[0].grid(axis='y', alpha=0.3)

# Add count and percentage labels
for i, (sent, count) in enumerate(sentiment_counts.items()):
    pct = (count / len(df)) * 100
    axes[0].text(i, count + 200, f'{count:,}\n({pct:.1f}%)', 
                ha='center', va='bottom', fontweight='bold', fontsize=11)

# Donut chart
wedges, texts, autotexts = axes[1].pie(sentiment_counts.values, labels=sentiment_counts.index,
                                        autopct='%1.1f%%', startangle=90, colors=colors,
                                        wedgeprops=dict(width=0.5, edgecolor='black', linewidth=1.5))

axes[1].set_title('Sentiment Proportion', fontweight='bold', fontsize=14)

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(12)

plt.tight_layout()
plt.show()

print("✅ Visualization complete!")

## 4. Sentiment by Product Category

In [ ]:
# Analyze sentiment per category
category_sentiment = pd.crosstab(df['topic'], df['label'], normalize='index') * 100
category_sentiment = category_sentiment.reindex(columns=['negative', 'neutral', 'positive'])

# Sort by positive sentiment percentage
category_sentiment_sorted = category_sentiment.sort_values('positive', ascending=False)

print("🏆 Product Categories by Positive Sentiment (Top 15):\n")
print(category_sentiment_sorted.head(15).round(2))

In [ ]:
# Visualize sentiment by top categories
top_categories = category_counts.head(12).index
category_sent_top = category_sentiment.loc[top_categories]

fig, ax = plt.subplots(figsize=(14, 8))

x = np.arange(len(category_sent_top))
width = 0.25

bars1 = ax.bar(x - width, category_sent_top['negative'], width, 
              label='Negative', color='#ef5350', edgecolor='black', linewidth=1)
bars2 = ax.bar(x, category_sent_top['neutral'], width, 
              label='Neutral', color='#ffa726', edgecolor='black', linewidth=1)
bars3 = ax.bar(x + width, category_sent_top['positive'], width, 
              label='Positive', color='#66bb6a', edgecolor='black', linewidth=1)

ax.set_xlabel('Product Category', fontweight='bold')
ax.set_ylabel('Percentage of Reviews (%)', fontweight='bold')
ax.set_title('Sentiment Distribution Across Top 12 Product Categories', fontweight='bold', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(category_sent_top.index, rotation=45, ha='right')
ax.legend(loc='upper right')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualization complete!")

## 5. Word Clouds - Key Themes by Sentiment

**Note:** This generates large visualizations showing the most common words in each sentiment category.

In [ ]:
# Generate word clouds for each sentiment
fig, axes = plt.subplots(3, 1, figsize=(16, 18))

sentiments = [
    ('positive', 'Greens', '🎉 Positive Reviews'),
    ('neutral', 'Oranges', '😐 Neutral Reviews'),
    ('negative', 'Reds', '😞 Negative Reviews')
]

for idx, (label, colormap, title) in enumerate(sentiments):
    # Get text for this sentiment
    texts = df[df['label'] == label]['text']
    text = ' '.join(texts.fillna(''))
    
    # Generate word cloud
    wordcloud = WordCloud(width=1400, height=400,
                         background_color='white',
                         colormap=colormap,
                         max_words=100,
                         relative_scaling=0.5,
                         min_font_size=10).generate(text)
    
    axes[idx].imshow(wordcloud, interpolation='bilinear')
    axes[idx].set_title(f'{title} - Most Common Words', fontsize=16, fontweight='bold', pad=20)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("✅ Word clouds generated successfully!")

## 6. Summary Insights

In [ ]:
# Calculate text statistics
df['text_length'] = df['text'].str.len()
df['word_count'] = df['text'].str.split().str.len()

print("\n" + "="*80)
print("📊 KEY INSIGHTS - CLOTHING BRAND SENTIMENT ANALYSIS")
print("="*80)

# 1. Overall sentiment
pos_pct = (sentiment_counts['positive'] / len(df)) * 100
print(f"\n1️⃣  OVERALL SENTIMENT:")
print(f"   {pos_pct:.1f}% of reviews are POSITIVE - High customer satisfaction")
print(f"   Imbalance ratio: {sentiment_counts['positive'] / sentiment_counts['negative']:.1f}:1")

# 2. Top performing categories
best_cat = category_sentiment.sort_values('positive', ascending=False).head(1)
print(f"\n2️⃣  BEST PERFORMING CATEGORY:")
print(f"   '{best_cat.index[0]}' - {best_cat['positive'].values[0]:.1f}% positive")

# 3. Categories needing attention
worst_cat = category_sentiment.sort_values('negative', ascending=False).head(1)
print(f"\n3️⃣  NEEDS ATTENTION:")
print(f"   '{worst_cat.index[0]}' - {worst_cat['negative'].values[0]:.1f}% negative")

# 4. Review characteristics
print(f"\n4️⃣  REVIEW CHARACTERISTICS:")
print(f"   Average review length: {df['text_length'].mean():.0f} characters")
print(f"   Average word count: {df['word_count'].mean():.0f} words")

print("\n" + "="*80)
print("\n✅ Analysis complete! You can now edit and run any cells to explore further.")